In [39]:
from torchvision.io.image import read_image
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image

# Step 1: Initialize model with the best available weights
weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model.eval()

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [40]:
img = read_image("pic.jpg")

# Step 3: Apply inference preprocessing transforms
batch = [preprocess(img)]

# Step 4: Use the model and visualize the prediction
prediction = model(batch)[0]
labels = [weights.meta["categories"][i] for i in prediction["labels"]]
box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                          labels=labels,
                          width=2, font_size=40)
im = to_pil_image(box.detach())
# im.show()

/opt/conda/lib/python3.7/site-packages/torchvision/utils.py:237: UserWarning: Argument 'font_size' will be ignored since 'font' is not set.
  warnings.warn("Argument 'font_size' will be ignored since 'font' is not set.")


In [42]:
prediction

{'boxes': tensor([[713.7922, 315.7549, 780.6686, 500.2391],
         [354.8589, 307.7073, 442.4601, 550.1827],
         [289.8649, 324.4239, 366.3903, 557.0024],
         [785.5298, 308.7691, 836.4589, 417.9224],
         [190.3003, 321.3901, 235.2892, 405.8893],
         [672.7024, 315.2165, 717.3039, 465.8602],
         [529.8246, 315.5521, 567.9066, 435.4109],
         [427.1546, 315.2697, 468.4995, 441.8229],
         [496.6395, 323.1866, 526.8000, 407.4753],
         [841.3839, 368.3748, 905.0833, 481.2395],
         [187.3358, 316.2392, 210.4650, 379.7055],
         [910.3627, 406.3258, 951.7915, 491.0837],
         [555.4110, 339.8068, 571.0477, 376.2867],
         [573.6293, 321.8709, 592.3687, 379.0994],
         [107.5550, 274.2000, 125.8037, 306.1205],
         [603.1185, 356.7075, 653.1871, 405.5475],
         [338.5075, 319.7022, 369.6255, 370.2662],
         [701.1111, 336.0122, 731.7516, 458.4563],
         [256.6582, 328.0870, 272.7791, 372.7920],
         [131.0138, 27

Actual code

In [52]:
CUSTOM_PREDICTOR_IMAGE_URI = "gcr.io/object-detection-365207/pytorch_predict_maskrcnn"
print(f"CUSTOM_PREDICTOR_IMAGE_URI = {CUSTOM_PREDICTOR_IMAGE_URI}")

CUSTOM_PREDICTOR_IMAGE_URI = gcr.io/object-detection-365207/pytorch_predict_maskrcnn


In [55]:
!docker build --tag=$CUSTOM_PREDICTOR_IMAGE_URI .

Sending build context to Docker daemon  698.6MB
Step 1/10 : FROM pytorch/torchserve:latest-cpu
latest-cpu: Pulling from pytorch/torchserve

6f815738: Pulling fs layer 
32c68000: Pulling fs layer 
7973901e: Pulling fs layer 
17495cfe: Pulling fs layer 
3ef7caed: Pulling fs layer 
f9753c3c: Pulling fs layer 
d7900702: Pulling fs layer 
cfa3ac56: Pulling fs layer 
Digest: sha256:86fd1541435a7d4c55277d52a8ebff8fd5aa93c7f43dc2236bf11fe7d6d7662e[8A
Status: Downloaded newer image for pytorch/torchserve:latest-cpu
 ---> af91330a97bd
Step 2/10 : COPY ./maskrcnn.mar /home/model-server/model-store/
 ---> 82288f064acc
Step 3/10 : USER root
 ---> Running in 8dab70e06fde
Removing intermediate container 8dab70e06fde
 ---> 1f3a838907f4
Step 4/10 : RUN printf "\nservice_envelope=json" >> /home/model-server/config.properties
 ---> Running in c15199a12374
Removing intermediate container c15199a12374
 ---> 3b177b80efdf
Step 5/10 : RUN printf "\ninference_address=http://0.0.0.0:7080" >> /home/model-server/

In [56]:
!docker run -t -d --rm -p 7080:7080 --name=maskrcnn $CUSTOM_PREDICTOR_IMAGE_URI

9240173c433bb17afa72245c80080ffe2ef2f5fab74f0a0122ba6e81a06cadc0


In [2]:
!docker push $CUSTOM_PREDICTOR_IMAGE_URI

Using default tag: latest
The push refers to repository [gcr.io/object-detection-365207/pytorch_predict_maskrcnn]

1240ae60: Preparing 
55fcad0f: Preparing 
702911bb: Preparing 
79bbf065: Preparing 
bf18a086: Preparing 
da41ec4a: Preparing 
d007c81a: Preparing 
c83319d2: Preparing 
005ec070: Preparing 
c8ae3daf: Preparing 
e9bfffc1: Preparing 
51f4d794: Preparing 
c8ae3daf: Pushed   881.4MB/867.9MBlatest: digest: sha256:7325c1ac9bd9c196203fc951ed281d6260138f1d515328d35262cebdf942a51d size: 3032


In [4]:
from google.cloud import aiplatform

APP_NAME = 'maskrcnn'
VERSION = 1
CUSTOM_PREDICTOR_IMAGE_URI = "gcr.io/object-detection-365207/pytorch_predict_maskrcnn"

model_display_name = f"{APP_NAME}-v{VERSION}"
model_description = "Object detector"

MODEL_NAME = APP_NAME
health_route = "/ping"
predict_route = f"/predictions/{MODEL_NAME}"
serving_container_ports = [7080]

model = aiplatform.Model.upload(
    display_name=model_display_name,
    description=model_description,
    serving_container_image_uri=CUSTOM_PREDICTOR_IMAGE_URI,
    serving_container_predict_route=predict_route,
    serving_container_health_route=health_route,
    serving_container_ports=serving_container_ports)

model.wait()

print(model.display_name)
print(model.resource_name)

Creating Model
Create Model backing LRO: projects/73250724104/locations/us-central1/models/3281677171050938368/operations/8337797128139046912
Model created. Resource name: projects/73250724104/locations/us-central1/models/3281677171050938368@1
To use this Model in another session:
model = aiplatform.Model('projects/73250724104/locations/us-central1/models/3281677171050938368@1')
maskrcnn-v1
projects/73250724104/locations/us-central1/models/3281677171050938368


In [5]:
endpoint_display_name = f"{APP_NAME}-endpoint"
endpoint = aiplatform.Endpoint.create(display_name=endpoint_display_name)

Creating Endpoint
Create Endpoint backing LRO: projects/73250724104/locations/us-central1/endpoints/4349496475667922944/operations/8682322499632889856
Endpoint created. Resource name: projects/73250724104/locations/us-central1/endpoints/4349496475667922944
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/73250724104/locations/us-central1/endpoints/4349496475667922944')


In [8]:
traffic_percentage = 100
machine_type = "n1-standard-2"
deployed_model_display_name = model_display_name
min_replica_count = 1
max_replica_count = 1
sync = True

model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=deployed_model_display_name,
    machine_type=machine_type,
    traffic_percentage=traffic_percentage,
    sync=sync)

model.wait()

Deploying model to Endpoint : projects/73250724104/locations/us-central1/endpoints/4349496475667922944
Deploy Endpoint model backing LRO: projects/73250724104/locations/us-central1/endpoints/4349496475667922944/operations/4052059132742598656
Endpoint model deployed. Resource name: projects/73250724104/locations/us-central1/endpoints/4349496475667922944


In [28]:
import base64

# https://cloud.google.com/vertex-ai/docs/general/base64
# https://cloud.google.com/blog/topics/developers-practitioners/pytorch-google-cloud-how-deploy-pytorch-models-vertex-ai
def encode_image(image):
    with open(image, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
    return encoded_string

img = encode_image('pic.jpg')

instance = [{
    "data": {
        "b64": str(img.decode('utf-8'))
    }
}]


prediction = endpoint.predict(instances=instance)
print(f"Prediction response: \n\t{prediction}")

Prediction response: 
	Prediction(predictions=[[{'person': [357.8408813476562, 308.4928283691406, 442.2432556152344, 550.4656982421875], 'score': 0.9992156028747559}, {'person': [289.5211791992188, 323.190185546875, 364.8891296386719, 554.4758911132812], 'score': 0.9988885521888733}, {'score': 0.9986439347267151, 'person': [718.21826171875, 316.1445922851562, 781.630126953125, 499.7513427734375]}, {'person': [189.2773895263672, 321.1208190917969, 236.1860656738281, 406.2220153808594], 'score': 0.9976911544799805}, {'person': [786.704345703125, 305.03125, 834.47900390625, 416.757080078125], 'score': 0.9975436329841614}, {'person': [422.6703186035156, 315.9645080566406, 468.377197265625, 441.4744567871094], 'score': 0.9907553791999817}, {'score': 0.9888505339622498, 'person': [672.0850830078125, 320.6743774414062, 714.8658447265625, 462.603759765625]}, {'potted plant': [839.2882080078125, 365.1861877441406, 904.1807250976562, 481.9791870117188], 'score': 0.988602340221405}, {'person': [4